In [1]:
import csv
import requests
import re
from lxml import etree as ET
import asyncio

In [2]:
tree = ET.parse('../data/kind-catalog.xml')
root = tree.getroot()

CATALOG = {
    "categories": [],
    "products": []
}

VARIANT_MASTER_MAP = {}

for child in root:
    if child.tag == "product":
        master_id = child.attrib["product-id"]
        CATALOG["products"].append(master_id)
        
        for children in child:
            if children.tag == 'variations':
                for variants in children[1]:
                    variant_id = variants.attrib['product-id']
                    VARIANT_MASTER_MAP[variant_id] = master_id
                    
storefront_tree = ET.parse('../data/kind-storefront.xml')
storefront_root = tree.getroot()

for child in root:
    if child.tag == "category":
        CATALOG["categories"].append(child.attrib["category-id"])
    
reader = csv.DictReader(open('../data/redirects.csv', 'rt', encoding="utf-8-sig"))
dict_urls = []
for line in reader:
    dict_urls.append(line)
    
reader = csv.DictReader(open('../data/product-redirects.csv', 'rt', encoding="utf-8-sig"))
dict_product = []
for line in reader:
    dict_product.append(line)
    
reader = csv.DictReader(open('../data/static_redirects.csv', 'rt', encoding="utf-8-sig"))
static_redirects = []
for line in reader:
    static_redirects.append(line)

In [27]:
STATIC_REDIRECTS = []

for redirect in static_redirects:
    match_string = re.search('pid,(.*?)$', redirect['redirect'])
    
    if match_string != None:
        if match_string.group(1) in VARIANT_MASTER_MAP:
            re.sub('pid,(.*?)$', match_string.group(1), redirect['redirect'])
        else:
          continue
    else:
        STATIC_REDIRECTS.append(redirect['redirect'])

In [28]:
with open('../data/remaining_static_redirects.csv', 'w') as csv_file:
    writer = csv.writer(csv_file)
    for redirect in STATIC_REDIRECTS:
        writer.writerow([redirect])